<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/water_resource_management/module_02_surface_water_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap
import ee

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Loading the dataset

Here we will use the [JRC global surface water dataset](https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_GlobalSurfaceWater) from google earth engine

This data contains the spatio-temporal distribution of surface water. It is a single image with 7 bands, where each band contains unique information. Let’s use the occurrence band, which includes information on the frequency of the water present from 1984-2020. The pixel value ranges from 0-100, where 0 represents No trace of water in any year, and 100 represents water present in all 36 years.


In [2]:
## Area of interest

geojson = {
  "coordinates": [
    [
      [
        77.1200409,
        11.5324541
      ],
      [
        76.9923248,
        11.5062217
      ],
      [
        76.9916382,
        11.3467571
      ],
      [
        77.1529998,
        11.4261642
      ],
      [
        77.1200409,
        11.5324541
      ]
    ]
  ],
  "type": "Polygon"
}

geometry = ee.Geometry(geojson)

## Plot the surface water

In [3]:
image = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('occurrence')

Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['#deebf7','#9ecae1','#3182bd']
}

Map.addLayer(image.clip(geometry), vis_params, 'Surface Water')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…

## Create a water mask

The `max_extend` band contains values either 0 or 1 for each pixel, 1 - where water is detected and 0 - no water detected

- We will use this band to mask out the no water pixels from the image

    - max_extent band has values 0 or 1
    - We can remove the 0 values by masking the layer
    - Mask needs to have 0 or 1 values
    - 0 - remove the pixel
    - 1 - keep the pixel

### Brief about image masking

Masking is done when you want certain pixels to be set to `nodata` value.

When pixel value is set as no data value, those pixels will be exlcuded from further analysis like calculating total number of pixels or when visualizing on map

With `.updateMask()` function, it takes a mask image contain int values either 1 or 0, values with 0 is set to no data and 1 is considered for further analysis:

<img src='https://github.com/kavyajeetbora/end_to_end_gee_with_python/blob/master/images/Masking%20Images.PNG?raw=true' height=300/>

There is another method in google earth engine called `.selfMask()`. It is when the image we want to mask is itself containing a binary values 1 or 0 in each pixel, we can go for this method instead of creating a mask from the original image.

In [4]:
isWater = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('max_extent')

masked = isWater.selfMask()

Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}
Map.addLayer(masked.clip(geometry), vis_params, 'Surface Water')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…

## Seasonality

It denotes for how many months there was water in the given pixel. It is very useful to detect permenant water resource

In [5]:
image = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('seasonality')

perm_water_mask = image.gt(5)

perm_water = image.updateMask(perm_water_mask)

Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}
Map.addLayer(perm_water.clip(geometry), vis_params, 'Permenant Surface Water')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…

## Find lost/gain water bodies

In [6]:
image = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('transition')

## Select classes that shows water lost
lost_water_mask = image.eq(3).Or(image.eq(6))

lost_water = image.updateMask(lost_water_mask)

Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}
Map.addLayer(lost_water.clip(geometry), vis_params, 'Lost Surface Water')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…

find the total surface water gain:

In [7]:
image = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('transition')

## Select classes that shows water lost
gain_water_mask = image.eq(2).Or(image.eq(5))

gain_water = image.updateMask(gain_water_mask)

Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}
Map.addLayer(gain_water.clip(geometry), vis_params, 'Surface Water Gain')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…

## Calculate the total gain and total loss

In [8]:
## Calculate the area in km2
pixel_area = gain_water.multiply(ee.Image.pixelArea().divide(1e6))

total_gain_area = pixel_area.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = geometry,
    scale=30
).getInfo()['transition']

## Calculate the area in km2
pixel_area = lost_water.multiply(ee.Image.pixelArea().divide(1e6))

total_lost_area = pixel_area.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = geometry,
    scale=30
).getInfo()['transition']

print(f"Total gain in surface area since 1984: {total_gain_area:.2f} km2")
print(f"Total loss in surface area since 1984: {total_lost_area:.2f} km2")

Total gain in surface area since 1984: 88.31 km2
Total loss in surface area since 1984: 2.09 km2


## Calculate the yearly surface water area

In [9]:
image_collection = ee.ImageCollection("JRC/GSW1_4/YearlyHistory")\
.filter(ee.Filter.bounds(geometry))\
.filter(ee.Filter.eq('year', 2010))

image = image_collection.first()
isWaterImage = image.eq(2).Or(image.eq(3))

isWaterImage = isWaterImage.selfMask()

Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}
Map.addLayer(isWaterImage.clip(geometry), vis_params, 'Surface Water Gain')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…

## Locate a SubBasin

What is Basin ?

A river basin (also known as a watershed or drainage basin) is an area of land where precipitation accumulates and flows into a river

In [23]:
basin = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_7")\
.filter(ee.Filter.eq('HYBAS_ID', 4071145720))

Map = geemap.Map()

Map.addLayer(basin)
Map.centerObject(geometry, zoom=9)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…

## Create a Surface Water Map for the basin

In [22]:
surface_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('max_extent')

image = surface_water.selfMask()

Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}

basin_vis_params = {
    "width": 2,
    "lineType": "solid",
    "fillColorOpacity": 0,
}

Map.addLayer(basin, basin_vis_params, 'Basin Boundary')
Map.addLayer(image.clip(basin), vis_params, 'Surface water')
Map.centerObject(geometry, zoom=9)
Map

Map(center=[11.448665908257853, 77.06105630046073], controls=(WidgetControl(options=['position', 'transparent_…